Pass or fail based on critieria 
final grade prediction for values that do not have a final grade 
a knn (idk which one yet)

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary

In [43]:
student_performace = pd.read_csv('student_performance_updated_1000.csv')


In [44]:
#clean data
student_performace.columns = [s.strip().replace(' ','_') for s in student_performace.columns]
#descriptive statistics
print(student_performace.describe())

reduced_performace = student_performace.dropna()
reduced_performace = reduced_performace[
    (reduced_performace['StudyHoursPerWeek'] > 0) |
    (reduced_performace['AttendanceRate'] < 100) |
    (reduced_performace['StudyHoursPerWeek'] < (24 * 7)) |
    (reduced_performace['FinalGrade'] <= 100)
]
x = reduced_performace.drop(columns=['FinalGrade'])
y = reduced_performace['FinalGrade']
reduced_performace.head()

reduced_performace= pd.get_dummies(reduced_performace,drop_first=True,dtype=int,columns=['ParentalSupport','Gender'])
print(reduced_performace.columns)

         StudentID  AttendanceRate  StudyHoursPerWeek  PreviousGrade  \
count   960.000000      960.000000         950.000000     967.000000   
mean   5416.019792       85.510417          17.630526      77.598759   
std    2653.748319        7.332125           6.272132      10.006640   
min       1.000000       70.000000           8.000000      60.000000   
25%    3113.500000       82.000000          12.000000      70.000000   
50%    5396.500000       88.000000          18.000000      78.000000   
75%    7754.750000       91.000000          22.000000      86.000000   
max    9998.000000       95.000000          30.000000      90.000000   

       ExtracurricularActivities  FinalGrade  Study_Hours  Attendance_(%)  
count                 957.000000  960.000000   976.000000      959.000000  
mean                    1.520376   80.030208     2.406967       77.248175  
std                     1.046439    9.493652     1.620267       19.298148  
min                     0.000000   62.000000   

In [45]:
scaler = MinMaxScaler()

features = ['StudyHoursPerWeek', 'AttendanceRate', 'PreviousGrade','Attendance_(%)','ExtracurricularActivities','Study_Hours']
reduced_performace_norm = reduced_performace.copy()
reduced_performace_norm[features] = scaler.fit_transform(reduced_performace[features])
X = pd.DataFrame(reduced_performace_norm, columns=features)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [46]:
scaler_z = StandardScaler()
reduced_performace_std = reduced_performace.copy()
reduced_performace_std[features] = scaler.fit_transform(reduced_performace[features])
reduced_performace_std.head()
X = pd.DataFrame(reduced_performace_norm, columns=features)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)